In [2]:
import torch
import torchvision
from torch.utils.data import DataLoader


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_data = torchvision.datasets.CIFAR10(root='./dataset', train=True, download=True, transform=torchvision.transforms.ToTensor())
test_data = torchvision.datasets.CIFAR10(root='./dataset', train=False, download=True, transform=torchvision.transforms.ToTensor())
train_data_size = len(train_data)
test_data_size = len(test_data)
print(f'训练数据集的长度{train_data_size}')
print(f'测试数据集的长度{test_data_size}')

100.0%


Extracting ./dataset\cifar-10-python.tar.gz to ./dataset
Files already downloaded and verified
训练数据集的长度50000
测试数据集的长度10000


In [4]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_data, batch_size=64)
test_loader = DataLoader(test_data, batch_size=64)


In [6]:
class model(torch.nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.model1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=5, padding=2),
            torch.nn.MaxPool2d(2),
            torch.nn.Conv2d(32, 32, kernel_size=5, padding=2),
            torch.nn.MaxPool2d(2),
            torch.nn.Conv2d(32, 64, kernel_size=5, padding=2),
            torch.nn.MaxPool2d(2),
            torch.nn.Flatten(),
            torch.nn.Linear(1024, 64),
            torch.nn.Linear(64, 10),
        )

    def forward(self, x):
        x = self.model1(x)
        return x

In [7]:
model = model()
model = model.to(device)

In [8]:
loss_fn = torch.nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [9]:
total_train_step = 0
total_test_step = 0
epoch =10

In [15]:

import time


start_time = time.time()
for i in range(epoch):
    print(f'-----------第{i + 1}轮训练开始----------')
    for data in train_loader:
        img,label = data
        img = img.to(device)
        label = label.to(device)
        output = model(img)
        loss = loss_fn(output, label)
        optimizer.zero_grad() #optimizer.zero_grad() 清空优化器的梯度
        loss.backward() #loss.backward() 计算损失相对于模型参数的梯度。
        optimizer.step() #optimizer.step() 更新模型参数。

        total_train_step += 1
        if total_train_step % 100 == 0:
            end_time = time.time()
            print(end_time - start_time)
            print(f'训练次数{total_train_step}，Loss：{loss.item()}')

    model.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_loader:
            img, label = data
            img = img.to(device)
            label = label.to(device)
            output = model(img)
            loss = loss_fn(output, label)
            total_test_loss += loss.item()
            accuracy = (output.argmax(1) == label).sum()
            total_accuracy += accuracy
        print(f'整体测试集上的loss：{total_test_loss}')
        print(f'整体测试集上的正确率：{total_accuracy / test_data_size}')

        total_test_step += 1

        torch.save(model.state_dict(), f'wt{i}.pth')
        print('模型已保存')


-----------第1轮训练开始----------
5.349006652832031
训练次数100，Loss：2.2901556491851807
10.488433599472046
训练次数200，Loss：2.2801098823547363
15.50047516822815
训练次数300，Loss：2.2424745559692383
20.63086700439453
训练次数400，Loss：2.147754669189453
25.63570761680603
训练次数500，Loss：2.031214952468872
30.608842849731445
训练次数600，Loss：1.996639609336853
35.41980051994324
训练次数700，Loss：2.022691249847412
整体测试集上的loss：318.85076904296875
整体测试集上的正确率：0.2754000127315521
模型已保存
-----------第2轮训练开始----------
44.34820747375488
训练次数800，Loss：1.9004418849945068
49.19538474082947
训练次数900，Loss：1.8871550559997559
54.14150667190552
训练次数1000，Loss：1.9384347200393677
59.01002311706543
训练次数1100，Loss：1.9878665208816528
63.82973837852478
训练次数1200，Loss：1.7271331548690796
68.84346389770508
训练次数1300，Loss：1.693472146987915
73.92446994781494
训练次数1400，Loss：1.7472727298736572
78.9309093952179
训练次数1500，Loss：1.845133662223816
整体测试集上的loss：297.33985805511475
整体测试集上的正确率：0.32019999623298645
模型已保存
-----------第3轮训练开始----------
87.92960262298584
训练次数1600，